In [484]:
import ipywidgets as widgets
from IPython.display import display

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings

warnings.filterwarnings('ignore')

# Modeling Incidence

In this notebook we present modeling of two two metrics of cancer incidence number of tumors located **insitu** per 1000 of totally diagnozed tumors (insitu means that abnormal cells have been found in their place of origin but have not spread to nearby tissues). And number of diagnozed **multiple tumors** per 1000 cases

The following model must reflect ongoing reforms in Ukrainian medical system as well as COVID19 outbreak and full-scale invasion in 2022

In [485]:
st_df = pd.read_csv('./final_dataset/stage_incidence_features.csv')

In [486]:
st_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   year                      312 non-null    float64
 1   region                    312 non-null    object 
 2   mtumors                   312 non-null    float64
 3   syncmtumors               312 non-null    float64
 4   insitu                    312 non-null    float64
 5   ncervix                   312 non-null    float64
 6   nhospotal_pht             312 non-null    float64
 7   nbeds_pht                 312 non-null    float64
 8   ybeds_pht                 312 non-null    float64
 9   nill_pht                  312 non-null    float64
 10  nvillage_ill_pht          312 non-null    float64
 11  bed_days_pht              312 non-null    float64
 12  dvisits_pht               312 non-null    float64
 13  hvisits_pht               312 non-null    float64
 14  ndoctors_p

In [487]:
st_df = st_df.drop(columns=['tincidence_pht', 'mtumors', 'syncmtumors', 'insitu', 'ncervix', 'nvillage_ill_pht',
                            'hvisits_pht', 'ndialysis_pht', 'ndialysis_pht', 'nbacter_pht', 'polluted_dumps', 
                            'not_cleaned_dumps', 'dumps_not_cleaned_enough', 'num_clearing_plants', 'nphysic_pht'])

In [488]:
st_df.columns

Index(['year', 'region', 'nhospotal_pht', 'nbeds_pht', 'ybeds_pht', 'nill_pht',
       'bed_days_pht', 'dvisits_pht', 'ndoctors_pht', 'nnursing_pht',
       'nx_ray_pht', 'nflurography_pht', 'nradiology_pht', 'nradlab_pht',
       'nсt_pht', 'ncardiogram_pht', 'ndiaglab_pht', 'nbiochem_pht',
       'ncyto_pht', 'nimun_pht', 'nendoscop_pht', 'nultrasound_pht', 'gdp',
       'air_pollution', 'cpi', 'population', 'mtumors_pht', 'syncmtumors_pht',
       'insitu_pht', 'insitu_pti', 'mtumors_pti'],
      dtype='object')

## Events that need to be accounted

1. **2012** - start of reorganization of medical system
2. **2014** - moratorium on reargonization to same existing medical system at the start of the war
3. **2015** - withdrawal of the moratorium 
4. **2017** - **2018** - creation of medical districts
5. **2018** - creation of National Health Service of Ukraine
6. **2020** - out break
7. **2022** - start of full-scale invasion

To model these events variables **reorg**, **morat**, **mdistrict**, **nhsu**, **cov**, **inv** will be added. 
Also interaction varables must be added

In [489]:
st_df['reorg'] = ((st_df.year >= 2012) & (st_df.year < 2014)).astype(int)
# st_df['morat'] = ((st_df.year >= 2014) & (st_df.year < 2015)).astype(int)
st_df['inner'] = ((st_df.year >= 2014) & (st_df.year < 2018)).astype(int)
st_df['nhsu'] = ((st_df.year >= 2018) & (st_df.year < 2019)).astype(int)
st_df['cov'] = ((st_df.year >= 2020) & (st_df.year < 2022)).astype(int)
st_df['inv'] = (st_df.year >= 2022).astype(int)

In [490]:

equipment_variables = [
    'nhospotal_pht', 'nbeds_pht', 'nx_ray_pht', 'nflurography_pht',
    'nradiology_pht', 'nradlab_pht', 'nсt_pht', 'ncardiogram_pht', 'ndiaglab_pht',
    'nbiochem_pht', 'ncyto_pht', 'nimun_pht', 'nendoscop_pht', 'nultrasound_pht',
]

personnel_variables = ['ndoctors_pht', 'nnursing_pht']

illness_variables = ['nill_pht']

environmental_variables = [
    'air_pollution'
]

dummies = ['reorg', 'inner', 'nhsu', 'cov']

all_explanatory_variables = equipment_variables + personnel_variables + illness_variables + environmental_variables

all_dependent_variables = ['mtumors_pti']

Also interaction terms must be added

1. Equipment variables must have interaction with **reorg**, **mdistrict**, **nhsu**
2. Personnel varaibles must have interaction with **reorg**, **mdistrict**, **nhsu**, **cov**, **inv**
3. Illness variables must have interaction with **reorg**, **mdistrict**, **nhsu**, **cov**, **inv** 


In [491]:
def add_interactions(df, interactions: list[str], x_names: list[str]):
    interaction_names = []
    for interaction in interactions:
        for x_name in x_names:
            interaction_name = f"{interaction}_{x_name}"
            df[interaction_name] = df[interaction] * df[x_name]
            interaction_names.append(interaction_name)
    return df, interaction_names 

# st_df, _ = add_interactions(st_df, ['reorg', "morat", 'inner', 'mdistrict', 'nhsu', 'cov'], equipment_variables)
# st_df, _ = add_interactions(st_df, ['reorg', "morat", 'inner', 'mdistrict', 'nhsu', 'cov'], personnel_variables)
# st_df, _ = add_interactions(st_df, ['reorg', "morat", 'inner', 'mdistrict', 'nhsu', 'cov'], illness_variables)

## Trend account

In [492]:
st_df['t'] = st_df['year'] - 2008

In [493]:

def coupled_detrend(df: pd.DataFrame, x_names: list[str]):
    df = df.copy()
    years = sorted(df.year.unique())
    for x_name in x_names:
        for year in years:
            x_at_year = df[df.year == year][x_name]
            df.loc[df.year == year, x_name] = x_at_year - x_at_year.mean()
    
    return df


## Quick fixes

In [494]:
st_df = st_df[st_df.year <= 2021]
st_df = st_df[~st_df.region.isin(['Донецька', 'Луганська'])]

## Model selection

In [495]:
def select_best_model_for(df: pd.DataFrame, target: str, ommit: list[str]) -> tuple[sm.OLS, list[str]]:
    predictors = df.drop(columns=ommit + [target], errors="ignore")

    predictors = predictors.select_dtypes(include=["number"])
    
    X = predictors.copy()
    y = df[target]
    
    X = sm.add_constant(X)
    
    best_model = sm.OLS(y, X).fit()

    best_aic = np.inf

    to_drop = None

    while len(X.columns) > 0:
        aic_not_changed = True
            
        for col in X.columns:
            temp_X = X.drop(col, axis = 1)
            temp_model = sm.OLS(y, temp_X).fit()
            if temp_model.aic < best_aic:
                best_aic = temp_model.aic
                best_model = temp_model
                to_drop = col
                aic_not_changed = False

        if aic_not_changed:
            break

        X = X.drop(to_drop, axis = 1)
    
    return best_model



In [496]:
ommit = ["age_group", "year", "region", "category",  'mtumors_pht', 'syncmtumors_pht',
         'insitu_pht', 'insitu_pti', 'mtumors_pti', 'ncervix', 'mtumors', 'syncmtumors', 'insitu', 
         'gdp', 'mtumors_pti', 'tincidence', 'cpi', 'ybeds_pht', 'population']

In [497]:
def get_interpretation_table(params, explanatory_names):

    interpretation_table = {dummy: [] for dummy in dummies}
    interpretation_table.update({"base": None})

    interpretation_table["base"] = [params.get(x_name, 0) for x_name in explanatory_names]

    for dummy in dummies:
        for x_name in explanatory_names:
            total_impact = params.get(x_name, 0) + params.get(f"{dummy}_{x_name}", 0)
            interpretation_table[dummy].append(total_impact)
        
    return pd.DataFrame(interpretation_table, index=explanatory_names)

### Simple model

In [512]:
explanatory_variables = ['ndiaglab_pht', 'ndoctors_pht', 'dvisits_pht',
                        'nсt_pht', 'nimun_pht', 'nradlab_pht', 
                        'nbiochem_pht', 'ncyto_pht',
                        'air_pollution']

dummies = ['reorg','inner', 'nhsu']

variables = explanatory_variables + dummies
df = st_df[variables + all_dependent_variables + ['t', 'year', 'region']]
df['dvisits_pht'] /= df['ndoctors_pht']

variables += ['mdevindex']

df['mdevindex'] = st_df[['nx_ray_pht', 'nflurography_pht', 'nultrasound_pht']].min(axis=1)

df, interaction_names = add_interactions(df, dummies, explanatory_variables + ['mdevindex'])
df = coupled_detrend(df, variables + all_dependent_variables)


## Multiple regression

In [513]:
y = df['mtumors_pti']
X = sm.add_constant(df[explanatory_variables + interaction_names + ["mdevindex"]])

mtumors_model = sm.OLS(y, X).fit()

print(mtumors_model.rsquared_adj)
get_interpretation_table(mtumors_model.params, explanatory_variables + ['mdevindex'])

0.26479188966824685


,reorg,inner,nhsu,base
ndiaglab_pht,-1.843131,-2.790599,-2.467293,-1.091366
ndoctors_pht,0.024071,0.185535,0.257088,0.086117
dvisits_pht,0.037403,0.018648,0.013974,0.021357
nсt_pht,-12.714592,-12.363221,-27.176723,-1.912866
nimun_pht,8.936006,-26.834252,-39.551092,4.179101
nradlab_pht,117.549558,80.002736,35.663048,49.988002
nbiochem_pht,-54.929574,10.035692,39.627144,8.258671
ncyto_pht,-1.635064,19.909858,-20.510979,-17.869440
air_pollution,0.014000,0.019455,0.013624,-0.010899
mdevindex,-1.432678,-1.687806,-1.610748,2.343953


In [500]:
print(mtumors_model.summary())

                            OLS Regression Results                            
Dep. Variable:            mtumors_pti   R-squared:                       0.368
Model:                            OLS   Adj. R-squared:                  0.265
Method:                 Least Squares   F-statistic:                     3.566
Date:                Thu, 17 Apr 2025   Prob (F-statistic):           5.02e-10
Time:                        01:50:18   Log-Likelihood:                -1063.9
No. Observations:                 286   AIC:                             2210.
Df Residuals:                     245   BIC:                             2360.
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.0005    

## Tests

In [501]:
import pandas as pd
import statsmodels.api as sm

X = df[variables]  
X = sm.add_constant(X) 

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

         Variable       VIF
0           const  1.000000
1    ndiaglab_pht  1.483257
2    ndoctors_pht  1.667054
3     dvisits_pht  1.465658
4         nсt_pht  1.192950
5       nimun_pht  1.301153
6     nradlab_pht  1.263542
7    nbiochem_pht  1.170260
8       ncyto_pht  1.137471
9   air_pollution  1.365150
10          reorg       NaN
11          inner       NaN
12           nhsu       NaN
13      mdevindex  1.423069


In [514]:
correlation_matrix = df[variables].corr()[['nradlab_pht', 'ncyto_pht', 'nimun_pht']]
correlation_matrix

,nradlab_pht,ncyto_pht,nimun_pht
ndiaglab_pht,-0.076444,-0.034548,-0.213720
ndoctors_pht,-0.272179,-0.061878,-0.093671
dvisits_pht,-0.379083,-0.003426,-0.036281
nсt_pht,-0.122205,-0.224140,-0.257581
nimun_pht,0.163449,0.248837,1.000000
nradlab_pht,1.000000,0.133526,0.163449
nbiochem_pht,0.018636,0.104058,0.235500
ncyto_pht,0.133526,1.000000,0.248837
air_pollution,0.032704,-0.086083,-0.005905
reorg,NaN,NaN,NaN


In [503]:
from statsmodels.stats.stattools import durbin_watson

# residuals from your model
residuals = mtumors_model.resid  
dw_stat = durbin_watson(residuals)
print(f'Durbin-Watson statistic: {dw_stat}')

Durbin-Watson statistic: 2.262391756517175


In [504]:
from statsmodels.stats.diagnostic import acorr_ljungbox

# For raw time series or residuals
ljung_box = acorr_ljungbox(mtumors_model.resid, lags=[10], return_df=True)
print(ljung_box)

      lb_stat     lb_pvalue
10  60.903796  2.443577e-09
